Import requirements

In [1]:
import os

import pandas as pd
from datetime import datetime as dt

Here, we create the CFB Model Dataframes

In [2]:
class CFBDataframe:
    def __init__(self):
        # initialize dataframes with headers
        self.drives = [[], pd.DataFrame()]
        self.games = [[], pd.DataFrame()]
        self.lines = [[], pd.DataFrame()]
        self.player = [[], pd.DataFrame()]
        self.usage = [[], pd.DataFrame()]
        self.recruit = [[], pd.DataFrame()]

    def csv_to_df(self, df_type, season):
        # set file name
        file = "../data/{i}/{j}_{i}.csv".format(i=df_type, j=season)
        # import data with Windows encoding
        try:
            df = pd.read_csv(file, encoding='ANSI')
        # if not Windows, do Mac encoding
        except LookupError:
            df = pd.read_csv(file, encoding='ISO-8859-1')
        # set first row as headers
        df.rename(columns=df.iloc[0])
        # concatenate all years of data on dataframe type match
        if df_type == "drives":
            self.drives[0].append(df)
            self.drives[1] = pd.concat(self.drives[0])
        elif df_type == "games":
            self.games[0].append(df)
            self.games[1] = pd.concat(self.games[0])
        elif df_type == "lines":
            self.lines[0].append(df)
            self.lines[1] = pd.concat(self.lines[0])
        elif df_type == "player_season_stats":
            self.player[0].append(df)
            self.player[1] = pd.concat(self.player[0])
        elif df_type == "player_usage":
            self.usage[0].append(df)
            self.usage[1] = pd.concat(self.usage[0])
        elif df_type == "recruiting_players":
            self.recruit[0].append(df)
            self.recruit[1] = pd.concat(self.recruit[0])


Build the CFB dataframe with the scraped game data

In [3]:
cfb_df = CFBDataframe()
for year in range(2010, dt.today().year):
    for directory in next(os.walk('../data'))[1]:
        try:
            cfb_df.csv_to_df(directory, year)
        except FileNotFoundError:
            pass

Here, we initialize attributes and methods used to manipulate CFB Model

In [4]:
class CFBModel:
    def __init__(self, df):
        # create dictionary of dataframes
        self.data = {"drives": df.drives[1], "games": df.games[1], "lines": df.lines[1],
                     "player": df.player[1], "usage": df.usage[1], "recruit": df.recruit[1]}

    def home_favored(self):
        mean_home_points = pd.Series.mean(self.data["games"]["_home_points"])
        mean_away_points = pd.Series.mean(self.data["games"]["_away_points"])
        home_favored = mean_home_points - mean_away_points
        return home_favored

Create and perform operations on model

In [5]:
cfb_model = CFBModel(cfb_df)
home_favor = cfb_model.home_favored()
print("Home teams win by an average of {:.2f} points".format(home_favor))

Home teams win by an average of 7.30 points
